In [5]:
##5_IMPLEMENT THE CONTINUOUS BAG OF WORDS (CBOW) MODEL:::::::::::::::::
#importing libraries
from keras.preprocessing import text
####from keras.utils import np_utils
from tensorflow.keras.utils import to_categorical
from keras.preprocessing import sequence
from keras.utils import pad_sequences
import numpy as np
import pandas as pd

In [6]:
#taking random sentences as data
#a_data preparation:::::::::::
data = """Deep learning (also known as deep structured learning) is part of a broader family of machine learning methods based on artificial neural networks with representation learning. Learning can be supervised, semi-supervised or unsupervised. 
Deep-learning architectures such as deep neural networks, deep belief networks, deep reinforcement learning, recurrent neural networks, convolutional neural networks and Transformers have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design, medical image analysis, climate science, material inspection and board game programs, where they have produced results comparable to and in some cases surpassing human expert performance.
"""
dl_data = data.split()

In [7]:
#tokenization
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(dl_data)
word2id = tokenizer.word_index

word2id['PAD'] = 0
id2word = {v:k for k, v in word2id.items()}
wids = [[word2id[w] for w in text.text_to_word_sequence(doc)] for doc in dl_data]

vocab_size = len(word2id)
embed_size = 100
window_size = 2 

print('Vocabulary Size:', vocab_size)
print('Vocabulary Sample:', list(word2id.items())[:10])

Vocabulary Size: 75
Vocabulary Sample: [('learning', 1), ('deep', 2), ('networks', 3), ('neural', 4), ('and', 5), ('as', 6), ('of', 7), ('machine', 8), ('supervised', 9), ('have', 10)]


In [9]:
#generating (context word, target/label word) pairs
#b_generate training data:::::::::::::::::::::
def generate_context_word_pairs(corpus, window_size, vocab_size):
    context_length = window_size*2
    for words in corpus:
        sentence_length = len(words)
        for index, word in enumerate(words):
            context_words = []
            label_word   = []            
            start = index - window_size
            end = index + window_size + 1
            
            context_words.append([words[i] 
                                 for i in range(start, end) 
                                 if 0 <= i < sentence_length 
                                 and i != index])
            label_word.append(word)

            x = pad_sequences(context_words, maxlen=context_length)
            y = to_categorical(label_word, vocab_size)
            yield (x, y)
            
i = 0
for x, y in generate_context_word_pairs(corpus=wids, window_size=window_size, vocab_size=vocab_size):
    if 0 not in x[0]:
        # print('Context (X):', [id2word[w] for w in x[0]], '-> Target (Y):', id2word[np.argwhere(y[0])[0][0]])
    
        if i == 10:
            break
        i += 1

In [10]:
#model building
#c_train model:::::::::::::::
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda

cbow = Sequential()
cbow.add(Embedding(input_dim=vocab_size, output_dim=embed_size, input_length=window_size*2))
cbow.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(embed_size,)))
cbow.add(Dense(vocab_size, activation='softmax'))
cbow.compile(loss='categorical_crossentropy', optimizer='rmsprop')

print(cbow.summary())

# from IPython.display import SVG
# from keras.utils.vis_utils import model_to_dot

# SVG(model_to_dot(cbow, show_shapes=True, show_layer_names=False, rankdir='TB').create(prog='dot', format='svg'))



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 100)            7500      
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 75)                7575      
                                                                 
Total params: 15075 (58.89 KB)
Trainable params: 15075 (58.89 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [11]:
#d_output::::::::::::::::::
for epoch in range(1, 6):
    loss = 0.
    i = 0
    for x, y in generate_context_word_pairs(corpus=wids, window_size=window_size, vocab_size=vocab_size):
        i += 1
        loss += cbow.train_on_batch(x, y)
        if i % 100000 == 0:
            print('Processed {} (context, word) pairs'.format(i))

    print('Epoch:', epoch, '\tLoss:', loss)
    print()


Epoch: 1 	Loss: 433.63586044311523

Epoch: 2 	Loss: 429.4492964744568

Epoch: 3 	Loss: 426.3296756744385

Epoch: 4 	Loss: 423.2464065551758

Epoch: 5 	Loss: 420.7001123428345



In [12]:
weights = cbow.get_weights()[0]
weights = weights[1:]
print(weights.shape)

pd.DataFrame(weights, index=list(id2word.values())[1:]).head()

(74, 100)


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
deep,-0.061565,0.028339,-0.042624,-0.023644,-0.016360,-0.027904,-0.050471,-0.013451,-0.048513,0.027297,...,-0.025499,0.013613,-0.029110,0.014882,-0.010334,-0.029818,0.017548,-0.065349,-0.038923,0.022942
networks,-0.037503,0.006897,-0.011901,0.016327,-0.042708,0.009574,-0.049561,-0.013894,0.063422,-0.001082,...,0.053637,-0.012860,0.031825,0.033695,0.016332,0.036255,-0.026564,0.023170,0.029373,0.045916
neural,-0.022454,0.022092,0.008546,0.010227,-0.020346,0.032127,-0.030135,0.047904,0.002161,0.017754,...,0.007840,-0.038327,-0.002151,-0.000374,0.029217,0.016605,-0.010405,0.019069,0.000087,0.038671
and,-0.009054,0.001622,-0.044194,-0.046415,-0.043521,-0.007826,0.006262,-0.014754,-0.036314,0.023630,...,-0.012830,0.002307,0.023699,-0.023148,-0.007089,-0.041135,0.029090,-0.045059,-0.047056,-0.039597
as,0.010888,-0.014353,-0.011473,-0.042789,-0.039237,-0.020208,0.011241,-0.008553,0.026667,-0.037033,...,0.010929,-0.047658,0.028165,0.006525,-0.015929,-0.029302,0.021935,0.016221,0.000610,-0.046031


In [13]:
from sklearn.metrics.pairwise import euclidean_distances

distance_matrix = euclidean_distances(weights)
print(distance_matrix.shape)

similar_words = {search_term: [id2word[idx] for idx in distance_matrix[word2id[search_term]-1].argsort()[1:6]+1] 
                   for search_term in ['deep']}

similar_words

(74, 74)


{'deep': ['applied', 'or', 'supervised', 'can', 'produced']}